# MySQL exercise w/ Pokemon dataset

In [ ]:
# import libaries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style='darkgrid')
import mysql.connector

# connect to MySQL server
cnx = mysql.connector.connect(user='root',
                              host='127.0.0.1',
                              database='pokemon')
# create cursor
cs = cnx.cursor()

In [ ]:
# query no.1: the 3 most and 3 least common pokemon types
cs.execute('''
    WITH cte AS
        (SELECT COUNT(DISTINCT pokedex_no) total FROM type)
    (SELECT 
        type, COUNT(type) cnt, ROUND((COUNT(type)/total*100), 1) percentage 
    FROM type, cte GROUP BY type ORDER BY COUNT(type) DESC LIMIT 3)
    UNION
    (SELECT type, COUNT(type) cnt, ROUND((COUNT(type)/total*100), 1) percentage FROM type, cte
    GROUP BY type ORDER BY COUNT(type) LIMIT 3);
''')

In [ ]:
# fetch rows and column names
q1 = cs.fetchall()
q1_col = cs.column_names

In [ ]:
q1

In [ ]:
# visualize as dataframe
q1_df = pd.DataFrame(q1, columns=q1_col)
q1_df.type = [i.replace('\r', '') for i in q1_df.type]
q1_df

In [ ]:
# query no.2: The rarest abilities and their owners
cs.execute('''
    WITH cte AS (
        SELECT a.pokedex_no, name, abilities, COUNT(abilities) cnt
            FROM abilities a JOIN basic b 
            ON a.pokedex_no = b.pokedex_no GROUP BY abilities
                    )
        SELECT * FROM cte WHERE cnt = (SELECT MIN(cnt) FROM cte)
        GROUP BY abilities ORDER BY abilities;
''')

In [ ]:
# fetch returned rows and column names thereof
q2 = cs.fetchall()
q2_col = [i[0] for i in cs.description]

In [ ]:
# visualize as dataframe
q2_df = pd.DataFrame(q2, columns=q2_col)
q2_df.abilities = [i.replace('\r', '') for i in q2_df.abilities]
q2_df

In [ ]:
# query no.3: the strongest pokemon by stats, and make full sentences out of the resutls
cs.execute('''
    SELECT
        CONCAT(
            name, ' (No.', high.pokedex_no, ') has the ',
            GROUP_CONCAT(description SEPARATOR ' and the '), ' and ',
            CASE
                WHEN is_legendary = 0 THEN 'is not legendary.'
                ELSE 'is legendary.'
            END
        ) full_description
    FROM (
        WITH full_stats AS (
            SELECT
                s.pokedex_no, name, hp, attack, defense, sp_attack, sp_defense, speed,
                (attack + defense + sp_attack + sp_defense + speed) sum_of_stats
            FROM
                stats s JOIN basic b ON s.pokedex_no = b.pokedex_no
            )
        SELECT
            pokedex_no, name, 'highest HP' description
        FROM
            full_stats WHERE hp = (SELECT MAX(hp) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'highest attack' description
        FROM
            full_stats WHERE attack = (SELECT MAX(attack) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'highest defense' description
        FROM
            full_stats WHERE defense = (SELECT MAX(defense) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'highest SP attack' description
        FROM
            full_stats WHERE sp_attack = (SELECT MAX(sp_attack) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'highest SP defense' description
        FROM
            full_stats WHERE sp_defense = (SELECT MAX(sp_defense) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'highest speed' description
        FROM
            full_stats WHERE speed = (SELECT MAX(speed) FROM stats)
        UNION
        SELECT
            pokedex_no, name, 'overall highest stats' description
        FROM
            full_stats WHERE sum_of_stats = (SELECT MAX(sum_of_stats) FROM full_stats)
        ) high
    JOIN other o ON high.pokedex_no = o.pokedex_no
    GROUP BY name ORDER BY name;
''')

In [ ]:
# fetch all rows
q3 = cs.fetchall()

In [ ]:
# results
q3

In [ ]:
# query no.4: the most vulnerable pokemon by how they fare against moves of different types
cs.execute('''
    SELECT ag.pokedex_no, name, (
        bug + dark + dragon + electric + fairy +
        fight + fire + flying + ghost + grass +
        ground + ice + normal + poison + psychic +
        rock + steel + water
        ) ag_sum
    FROM against ag JOIN basic b
    ON ag.pokedex_no = b.pokedex_no
    ORDER BY ag_sum, pokedex_no;
''')

In [ ]:
# fetch all returned rows
q4 = cs.fetchall()
q4_col = [i[0] for i in cs.description]

In [ ]:
q4

In [ ]:
# visualize the result
q4_df = pd.DataFrame(q4, columns=q4_col)
q4_df

In [ ]:
# create temporary table resulted from query no.4
cs.execute('''
    CREATE TEMPORARY TABLE ag_sum
        SELECT ag.pokedex_no, name, (
            bug + dark + dragon + electric + fairy +
            fight + fire + flying + ghost + grass +
            ground + ice + normal + poison + psychic +
            rock + steel + water
            ) sum_ag
        FROM against ag JOIN basic b
        ON ag.pokedex_no = b.pokedex_no
        ORDER BY sum_ag, pokedex_no;
''')

In [ ]:
# create a summary that has type, sum of stats, sum of coefficients against moves & abilities
cs.execute('''CREATE TEMPORARY TABLE summary
    SELECT
        tbl1.pokedex_no, name, type, no_of_type,
        sum_stats, sum_ag, no_of_abilities, abilities, is_legendary
    FROM (
        SELECT 
            s.pokedex_no,
            SUM(hp + attack + defense + sp_attack + sp_defense + speed) sum_stats,
            sum_ag
        FROM stats s 
        JOIN ag_sum ag ON s.pokedex_no = ag.pokedex_no
        GROUP BY pokedex_no
    ) tbl1 
    JOIN (
        SELECT
            pokedex_no, 
            COUNT(abilities) no_of_abilities, 
            GROUP_CONCAT(abilities SEPARATOR ', ') abilities
        FROM abilities ab 
        GROUP BY pokedex_no
    ) tbl2 
    ON tbl1.pokedex_no = tbl2.pokedex_no
    JOIN (
        SELECT 
            pokedex_no, COUNT(type) no_of_type,
            GROUP_CONCAT(type SEPARATOR ', ') type
        FROM type 
        GROUP BY pokedex_no
    ) tbl3 
    ON tbl1.pokedex_no = tbl3.pokedex_no
    JOIN basic b ON tbl1.pokedex_no = b.pokedex_no
    JOIN other o ON tbl1.pokedex_no = o.pokedex_no
    ORDER by tbl1.pokedex_no;
''')

In [ ]:
# check the created temp table
cs.execute('SELECT * FROM summary;')
q5 = cs.fetchall()
q5_col = [i[0] for i in cs.description]

In [ ]:
# visualize the results
q5_df = pd.DataFrame(q5, columns=q5_col)
q5_df.type = [i.replace('\r', '') for i in q5_df.type]
q5_df.abilities = [i.replace('\r', '') for i in q5_df.abilities]
q5_df.sum_stats = q5_df.sum_stats.astype('int64')
q5_df.is_legendary = q5_df.is_legendary.astype(bool)
q5_df

## Exploratory Data Analysis

In [ ]:
# x: sum_stats
# y: sum_ag
# z: no_of_abilities
# 0: not legendary/single-typed
# 1: not legendary/double-typed
# 2: is legendary/single-typed
# 3: is legendary/double-typed
dt = {}
for i in ('x', 'y', 'z'):
    for j in range(4):    
        dt[f'{i}{j}'] = []
        
for i in q5_df.iloc:
    if i.is_legendary == False:
        if i.no_of_type == 1:
            dt['x0'].append(i.sum_stats)
            dt['y0'].append(i.sum_ag)
            dt['z0'].append(i.no_of_abilities)
        else:
            dt['x1'].append(i.sum_stats)
            dt['y1'].append(i.sum_ag)
            dt['z1'].append(i.no_of_abilities)
    else:
        if i.no_of_type == 1:
            dt['x2'].append(i.sum_stats)
            dt['y2'].append(i.sum_ag)
            dt['z2'].append(i.no_of_abilities)
        else:
            dt['x3'].append(i.sum_stats)
            dt['y3'].append(i.sum_ag)
            dt['z3'].append(i.no_of_abilities)

dt['m'] = ['.', 'x', '.', 'x'] # dot: single-typed, x: double-typed
dt['c'] = ['c', 'c', 'r', 'r'] # cyan: not legendary, red: legendary
dt['l'] = ['1-typed / not legendary',
           '2-typed / not legendary',
           '1-typed / legendary',
           '2-typed / legendary']

In [ ]:
# 3D scatterplot
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

for i in range(4):
    ax.scatter(dt[f'x{i}'], dt[f'y{i}'], dt[f'z{i}'], s=50,
               facecolor=dt['c'][i], marker=dt['m'][i], label=dt['l'][i])

ax.set_xlabel('sum_stats')
ax.set_ylabel('sum_ag')
ax.set_zlabel('no_of_abilities')
ax.set_title('Relationship between various factors')
ax.legend()
ax.view_init(10, 315)
plt.savefig('poke_3d_plot.png', bbox_inches='tight')
plt.show()

In [ ]:
# end the session
cnx.close()